In [1]:
import xlrd
import pandas as pd

In [2]:
def dateFromFilename(file='20200120_report.xlsx'):
    # The filename is always a date followed either by '_report.xlsx' or '_update.xlsx'
    date = file.replace('_report.xlsx', '').replace('_update.xlsx', '')
    return date

In [3]:
def openBCCfile(file = '20200120_report.xlsx'):
    date = dateFromFilename(file)
    data_dir = '~/projects/bcc_weekly_reports/data/'
    loc = (data_dir + file)

    print('Opening {}'.format(loc))
    wb = xlrd.open_workbook(loc)
    sheet = wb.sheet_by_name('COA Stats')
    print('Rows: {}, Cols: {}'.format(sheet.ncols, sheet.nrows))
    
    return wb, sheet

wb, sheet = openBCCfile(file = '20200120_report.xlsx')

Opening ~/projects/bcc_weekly_reports/data/20200120_report.xlsx
Rows: 4, Cols: 24


In [4]:
def importCOAdata(table='totals', file = '20200120_report.xlsx'):
    '''
    Return a df of one of the 3 tables found in the COA weekly report
    '''
    # Which table to import
    if table == 'totals':
        rows = range(1,3)
    elif table == 'product_category':
        rows = range(4,9)
    elif table == 'fail_category':
        rows = range(9,sheet.nrows-1)

    # Create dataframe from table
    data = []
    for row in rows:
        data.append(sheet.row_values(row))
    data_df = pd.DataFrame(data=data[1:], columns=data[0], dtype='int')
    
    # Create date column and make dtype=datetime
    date = dateFromFilename(file)
    data_df['Date'] = pd.to_datetime(date)
    
    return data_df


In [8]:
# Generate list of files
from os import listdir
data_dir2 = '../../data/'
files = [f for f in listdir(data_dir2) if '.xlsx' in f]

In [9]:
# Use file list to generate list of dates
dates=[]
for file in files: 
    dates.append(file.replace('_report.xlsx', '').replace('_update.xlsx',''))
dates = sorted(dates)

In [11]:
# Read all data into dataframes
totals = pd.DataFrame()
product_categories = pd.DataFrame()
fail_categories = pd.DataFrame()

for file in files: 
    # Read file
    wb, sheet = openBCCfile(file=file)
    # Read tables and save as dataframes
    totals = totals.append(importCOAdata(table='totals', file=file))
    product_categories = product_categories.append(importCOAdata(table='product_category', file=file))
    fail_categories = fail_categories.append(importCOAdata(table='fail_category', file=file))

Opening ~/projects/bcc_weekly_reports/data/20181113_report.xlsx
Rows: 4, Cols: 19
Opening ~/projects/bcc_weekly_reports/data/20190429_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20181203_report.xlsx
Rows: 4, Cols: 19
Opening ~/projects/bcc_weekly_reports/data/20190610_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20190527_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20191209_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20191007_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20190729_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20190923_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20190603_report.xlsx
Rows: 4, Cols: 24
Opening ~/projects/bcc_weekly_reports/data/20181029_report.xlsx
Rows: 4, Cols: 19
Opening ~/projects/bcc_weekly_reports/data/20181210_report.xlsx
Rows: 4, Cols: 19
Opening ~/projec

In [12]:
# Remove empty columns
del totals['']
del product_categories['']
del fail_categories['']

In [27]:
# Clean up totals df
del totals['Certificates of Analysis Received'] # This column is identical to 'Tested Batches'. Don't need both
totals['Percent Failed'] = 100* totals['Failed Batches'] / totals['Tested Batches']

KeyError: 'Certificates of Analysis Received'

In [15]:
# Save totals df
print('Saving totals.csv...')
totals.to_csv(path_or_buf='../../etl_data/totals.csv', index=False)

,Tested Batches,Failed Batches,Date,Percent Failed
0,20797,3164,2018-11-13,15.213733
0,46503,4679,2019-04-29,10.061716
0,23864,3373,2018-12-03,14.134261
0,52471,4906,2019-06-10,9.349927
0,50811,4831,2019-05-27,9.507784


In [ ]:
# Functions

# Clean up product_categories df

# Function to rename Columns. (Make new column from old, delete old column)
def removeCol(df, column, newname):
    try: 
        df[column]
    except: 
        return df
    else:
        df[newname] = df[column]
        del df[column]
        return df

# Some numeric values have unwanted characters (',') in them and are thus 'objects'
# Convert these to ints
def makeFloat(data):
    valid = '1234567890.' #valid characters for a float
    try: 
        ''.join(filter(lambda char: char in valid, data))
    except: 
        return data
    else: 
        return int(''.join(filter(lambda char: char in valid, data)))

# Sometimes unwanted characters appear in our strings (',', ':', etc)
# Remove specific string from 'Category' strings
def stringRemove(data, string):
    try: 
        data.replace(string,'')
    except: 
        return data
    else: 
        return data.replace(string,'')

In [17]:
# Clean up product_categories df

# Rename Columns
product_categories = removeCol(product_categories, 'Tested Batches By Category', newname='Category')
product_categories = removeCol(product_categories, 'Failed Batches By Category', newname='Failed Batches')

# Some values has ',' in them and are thus 'objects'
# Convert these to ints
product_categories['Failed Batches'] = product_categories['Failed Batches'].apply(makeFloat)
product_categories['Failed Batches'] = product_categories['Failed Batches'].astype('int')

# Remove specific string from 'Category' strings
product_categories['Category'] = product_categories['Category'].apply(stringRemove, args=':')
product_categories['Category'] = product_categories['Category'].apply(stringRemove, args=',')

# Calculated columns
product_categories['Percent Failed'] = 100* product_categories['Failed Batches'] / product_categories['Tested Batches']

# Aggregate for more normalizations
pc_totals_cols = ['Date', 'Tested Batches', 'Failed Batches']
pc_totals = product_categories[pc_totals_cols].groupby(by='Date').sum()/2
# Rename Aggregated Columns
pc_totals = removeCol(pc_totals, 'Failed Batches', newname='Total Failed')
pc_totals['Total Failed'] = pc_totals['Total Failed'].astype('int')
pc_totals = removeCol(pc_totals, 'Tested Batches', newname='Total Tested')
pc_totals['Total Tested'] = pc_totals['Total Tested'].astype('int')
# Merge with Category Data
product_categories = pc_totals.merge(product_categories, left_index=True, right_on='Date', how='right')
product_categories['Percent of Failures'] = 100 * product_categories['Failed Batches'] / product_categories['Total Failed']
product_categories['Percent Tested'] = 100 * product_categories['Tested Batches'] / product_categories['Total Tested']

# Save to file
print('Saving product_categories...')
product_categories.to_csv(path_or_buf='../../etl_data/product_categories.csv', index=False)

product_categories.head()

Saving product_categories...


,Total Failed,Total Tested,Tested Batches,Date,Category,Failed Batches,Percent Failed,Percent of Failures,Percent Tested
0,3164,20797,10415,2018-11-13,Flower,1029,9.879981,32.522124,50.079338
1,3164,20797,6464,2018-11-13,Inhalable\n(cartridges waxes etc.),1054,16.305693,33.312263,31.081406
2,3164,20797,3918,2018-11-13,Other\n(edibles tinctures topicals etc.),1081,27.590607,34.165613,18.839256
3,3164,20797,20797,2018-11-13,Total,3164,15.213733,100.000000,100.000000
0,4679,46503,23347,2019-04-29,Flower,1627,6.968775,34.772387,50.205363


In [19]:
# May want pivoted version of table later
pc_pivot = product_categories.pivot(index='Date', columns='Category')
# pc_pivot.swaplevel(axis=1)

pc_pivot.to_csv(path_or_buf='../../etl_data/pc_pivot.csv', index=True)

pc_pivot.head()

Total Failed                                     \
Category         Flower Inhalable\n(cartridges waxes etc.)   
Date                                                         
2018-10-09         2683                               2683   
2018-10-15         2804                               2804   
2018-10-22         2868                               2868   
2018-10-29         2985                               2985   
2018-11-05         3053                               3053   

                                                          Total Tested  \
Category   Other\n(edibles tinctures topicals etc.) Total       Flower   
Date                                                                     
2018-10-09                                     2683  2683        16049   
2018-10-15                                     2804  2804        16869   
2018-10-22                                     2868  2868        17833   
2018-10-29                                     2985  2985        18863   
2018-11-05                                     3053  3053        19707   

                                               \
Category   Inhalable\n(cartridges waxes etc.)   
Date                                            
2018-10-09                              16049   
2018-10-15                              16869   
2018-10-22                              17833   
2018-10-29                              18863   
2018-11-05                              19707   

                                                           Tested Batches  \
Category   Other\n(edibles tinctures topicals etc.)  Total         Flower   
Date                                                                        
2018-10-09                                    16049  16049           7993   
2018-10-15                                    16869  16869           8446   
2018-10-22                                    17833  17833           8978   
2018-10-29                                    18863  18863           9426   
2018-11-05                                    19707  19707           9829   

                                               ...  \
Category   Inhalable\n(cartridges waxes etc.)  ...   
Date                                           ...   
2018-10-09                               4991  ...   
2018-10-15                               5208  ...   
2018-10-22                               5483  ...   
2018-10-29                               5858  ...   
2018-11-05                               6138  ...   

                                     Percent Failed             \
Category   Other\n(edibles tinctures topicals etc.)      Total   
Date                                                             
2018-10-09                                29.853181  16.722741   
2018-10-15                                29.362364  16.622206   
2018-10-22                                28.736655  16.082544   
2018-10-29                                28.248114  15.824630   
2018-11-05                                27.700535  15.491957   

           Percent of Failures                                     \
Category                Flower Inhalable\n(cartridges waxes etc.)   
Date                                                                
2018-10-09           31.792769                          34.103615   
2018-10-15           32.738944                          33.594864   
2018-10-22           32.531381                          33.682008   
2018-10-29           32.428811                          33.701843   
2018-11-05           32.459876                          33.606289   

                                                                 \
Category   Other\n(edibles tinctures topicals etc.)       Total   
Date                                                              
2018-10-09                                34.103615  100.037272   
2018-10-15                                33.666191  100.000000   
2018-10-22                                33.786611  100.0000

In [20]:
# Clean up fail_categories df

#Rename columns
fail_categories = removeCol(fail_categories, 'Failed Batches By Category', newname='Failed Batches')
fail_categories = removeCol(fail_categories, '*Reasons For Failure', newname='Failure Reason')

# Remove ':'
fail_categories['Failure Reason'] = fail_categories['Failure Reason'].apply(stringRemove, args=':')
# Remove '*'
# Revisit later
fail_categories['Failure Reason'] = fail_categories['Failure Reason'].apply(stringRemove, args='*')

# Convert 'Failed Batches' to int
fail_categories['Failed Batches'] = fail_categories['Failed Batches'].apply(makeFloat)
fail_categories['Failed Batches'] = fail_categories['Failed Batches'].astype('int')

# Calculated columns
# 'Total Failed'
fc_totals = fail_categories.groupby(by='Date').sum()/2
fc_totals = removeCol(fc_totals, 'Failed Batches', newname='Total Failed')
fc_totals['Total Failed'] = fc_totals['Total Failed'].astype('int')
fail_categories = fc_totals.merge(fail_categories, left_index=True, right_on='Date', how='right')
fail_categories['Percent of Failures'] = 100 * fail_categories['Failed Batches'] / fail_categories['Total Failed']

print('Saving fail_categories.csv...')
fail_categories.to_csv(path_or_buf='../../etl_data/fail_categories.csv', index=False)

fail_categories.dtypes

Saving fail_categories.csv...


Total Failed                    int64
Date                   datetime64[ns]
Failed Batches                  int64
Failure Reason                 object
Percent of Failures           float64
dtype: object

In [22]:
# May want pivot version of table later
fc_pivot = fail_categories.pivot(index='Date', columns='Failure Reason', values='Failed Batches')
fc_pivot = fc_pivot.fillna(0)
fc_pivot = fc_pivot.astype('int')
fc_pivot.head()

Failure Reason,Cannabinoids,Foreign Material,Heavy Metals,Homogeneity,Injurious to Human Health,Label Claims,Microbial Impurities,Moisture,Mycotoxins,Pesticides,Residual Solvents,Total,Water Activity
Date,,,,,,,,,,,,,
2018-10-09,0,10,0,35,0,1751,201,40,0,602,130,2769,0
2018-10-15,0,10,0,35,0,1823,217,45,0,632,137,2899,0
2018-10-22,0,10,0,35,0,1867,224,45,0,645,145,2971,0
2018-10-29,0,10,0,35,0,1953,233,45,0,663,154,3093,0
2018-11-05,0,10,0,35,0,1979,249,50,0,672,167,3162,0


['Label Claims',
 'Pesticides',
 'Microbial Impurities',
 'Residual Solvents',
 'Homogeneity',
 'Foreign Material',
 'Moisture',
 'Heavy Metals',
 'Water Activity',
 'Cannabinoids',
 'Mycotoxins',
 'Injurious to Human Health']

In [24]:
for f in fail_reasons:
    fc_pivot['Percent {}'.format(f)] = fc_pivot[f] / fc_pivot['Total']
fc_pivot.head()

Failure Reason,Cannabinoids,Foreign Material,Heavy Metals,Homogeneity,Injurious to Human Health,Label Claims,Microbial Impurities,Moisture,Mycotoxins,Pesticides,...,Percent Microbial Impurities,Percent Residual Solvents,Percent Homogeneity,Percent Foreign Material,Percent Moisture,Percent Heavy Metals,Percent Water Activity,Percent Cannabinoids,Percent Mycotoxins,Percent Injurious to Human Health
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-09,0,10,0,35,0,1751,201,40,0,602,...,0.072589,0.046948,0.012640,0.003611,0.014446,0.0,0.0,0.0,0.0,0.0
2018-10-15,0,10,0,35,0,1823,217,45,0,632,...,0.074853,0.047258,0.012073,0.003449,0.015523,0.0,0.0,0.0,0.0,0.0
2018-10-22,0,10,0,35,0,1867,224,45,0,645,...,0.075395,0.048805,0.011781,0.003366,0.015146,0.0,0.0,0.0,0.0,0.0
2018-10-29,0,10,0,35,0,1953,233,45,0,663,...,0.075331,0.049790,0.011316,0.003233,0.014549,0.0,0.0,0.0,0.0,0.0
2018-11-05,0,10,0,35,0,1979,249,50,0,672,...,0.078748,0.052815,0.011069,0.003163,0.015813,0.0,0.0,0.0,0.0,0.0


In [25]:
fc_pivot.dtypes

Failure Reason
Cannabinoids                           int64
Foreign Material                       int64
Heavy Metals                           int64
Homogeneity                            int64
Injurious to Human Health              int64
Label Claims                           int64
Microbial Impurities                   int64
Moisture                               int64
Mycotoxins                             int64
Pesticides                             int64
Residual Solvents                      int64
Total                                  int64
Water Activity                         int64
Percent Label Claims                 float64
Percent Pesticides                   float64
Percent Microbial Impurities         float64
Percent Residual Solvents            float64
Percent Homogeneity                  float64
Percent Foreign Material             float64
Percent Moisture                     float64
Percent Heavy Metals                 float64
Percent Water Activity               flo

In [26]:
product_categories.tail()

,Total Failed,Total Tested,Tested Batches,Date,Category,Failed Batches,Percent Failed,Percent of Failures,Percent Tested
3,3310,22096,22096,2018-11-26,Total,3310,14.980087,100.000000,100.000000
0,4589,44017,22277,2019-04-15,Flower,1589,7.132917,34.626280,50.609992
1,4589,44017,14150,2019-04-15,Inhalable\n(cartridges waxes etc.),1598,11.293286,34.822401,32.146671
2,4589,44017,7590,2019-04-15,Other\n(edibles tinctures topicals etc.),1402,18.471673,30.551318,17.243338
3,4589,44017,44017,2019-04-15,Total,4589,10.425517,100.000000,100.000000
